In [1]:
import cv2 as cv
import numpy as np
import torch
import torch.nn as nn
import torchvision
import glob

In [2]:
imgs = glob.glob('*.jpg')

imgs

['angry.jpg', 'bored.jpg', 'happy.jpg', 'neutral.jpg', 'tired.jpg']

In [3]:
# Show all images
def show_imgs():
    for img in imgs:
        cv.imshow(img, cv.imread(img))
        cv.waitKey(0)
        cv.destroyWindow(img)

# Grayscale
def show_grayscale():
    for img in imgs:
        gray = cv.cvtColor(cv.imread(img), cv.COLOR_BGR2GRAY)
        cv.imshow(img, gray)
        cv.waitKey(0)
        cv.destroyWindow(img)